### CÉLULA 0: Importação de Bibliotecas Essenciais 🛠️

In [ ]:
!pip install -qU langchain # LangChain é útil para abstrair a interação com o LLM

In [ ]:
!pip install -qU llama-cpp-python
# Verifica se a instalação do llama-cpp-python foi bem-sucedida
try:
    from llama_cpp import Llama
    print("llama-cpp-python instalado com sucesso!")
except ImportError:
    print("Erro na instalação do llama-cpp-python. Verifique as saídas acima.")
    print("Se a instalação com CUDA falhou, tente comentar a primeira linha de instalação e descomentar a segunda.")

print("Dependências instaladas.")

llama-cpp-python instalado com sucesso!
Dependências instaladas.


In [ ]:
# Instala a biblioteca huggingface_hub para baixar modelos
!pip install -qU huggingface_hub

In [ ]:
!pip install -qU langchain-community # Necessário para LlamaCpp e outras integrações da LangChain

In [ ]:
from huggingface_hub import hf_hub_download
import os

model_name = "mistral-7b-instruct-v0.2.Q4_K_M.gguf" # Altere para o nome do novo arquivo GGUF
model_repo_id = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF" # Altere para o novo repositório
# Define o diretório onde o modelo será salvo
model_dir = "./models"
os.makedirs(model_dir, exist_ok=True) # Cria o diretório se não existir

# Caminho completo para o modelo
model_path = os.path.join(model_dir, model_name)

# Verifica se o modelo já existe para evitar download repetido
if not os.path.exists(model_path):
    print(f"Baixando o modelo {model_name} de {model_repo_id}...")
    hf_hub_download(
        repo_id=model_repo_id,
        filename=model_name,
        local_dir=model_dir,
        local_dir_use_symlinks=False # Importante para Colab
    )
    print("Download do modelo concluído!")
else:
    print(f"O modelo {model_name} já existe em {model_dir}. Pulando o download.")

print(f"Caminho do modelo: {model_path}")


O modelo mistral-7b-instruct-v0.2.Q4_K_M.gguf já existe em ./models. Pulando o download.
Caminho do modelo: ./models/mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Configura o callback para streaming da saída
#callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


llm = LlamaCpp(
    model_path=model_path, # Caminho para o modelo GGUF
    n_ctx=2048,           # Contexto máximo de tokens (ajuste conforme o modelo)
    n_gpu_layers=-1,      # Descarrega todas as camadas para a GPU (se disponível)
    n_threads=os.cpu_count(), # Usa o número de threads da CPU disponível
    temperature=0.3,      # Reduzido para tornar as respostas mais determinísticas
    max_tokens=100,       # Ajustado para garantir que a resposta completa seja gerada
    verbose=False          # Saída detalhada
)
#callback_manager=callback_manager, # Para ver a saída em tempo real
print("\nModelo LLM inicializado com sucesso!")

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized



Modelo LLM inicializado com sucesso!


### CÉLULA 1: Importação de Bibliotecas Essenciais 📚


In [ ]:
import os
import requests
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

### CÉLULA 2: Configuração do Webhook do Discord ⚙️


In [ ]:
# Substitua este URL pelo Webhook URL que você copiou do Discord
DISCORD_WEBHOOK_URL_PLACEHOLDER = "COLE_AQUI_SEU_WEBHOOK"

DISCORD_WEBHOOK_URL = os.getenv("DISCORD_WEBHOOK_URL", DISCORD_WEBHOOK_URL_PLACEHOLDER)

### CÉLULA 3: Cadeia de Análise PDFs (LLMChain) 🧠

---






In [ ]:
slide_summary_prompt = PromptTemplate.from_template(
    """Você é um assistente de estudo prestativo.
Crie um resumo conciso, com no máximo 20 linhas, do seguinte texto de slide.
O resumo deve focar nas ideias centrais, usar linguagem simples e ter um tom informativo e amigável para auxiliar estudantes.

Texto do Slide: "{texto_do_slide}"

Resumo:"""
)
# Remove o 'stop' para permitir um resumo mais longo
slide_summary_chain = slide_summary_prompt | llm | StrOutputParser()

### CÉLULA 4: Cadeia de Sugestão de Resposta ao Cliente (LLMChain) 💬


In [ ]:
key_concepts_prompt = PromptTemplate.from_template(
    """Analise o seguinte texto de slide e identifique os 3-5 conceitos mais importantes.
Liste cada conceito em uma linha separada, começando com um hífen.

Texto do Slide: "{texto_do_slide}"

Conceitos-Chave:"""
)
key_concepts_chain = key_concepts_prompt | llm | StrOutputParser()

### CÉLULA 5: Cadeia de Avaliação de Prioridade (LLMChain) 🚀


In [ ]:
study_questions_prompt = PromptTemplate.from_template(
    """Com base no seguinte texto de slide, crie 2-3 perguntas de estudo que ajudem os estudantes a testar seu entendimento do conteúdo.
As perguntas devem ser claras e focadas nos pontos principais.

Texto do Slide: "{texto_do_slide}"

Perguntas de Estudo:"""
)
study_questions_chain = study_questions_prompt | llm | StrOutputParser()

### CÉLULA 6: Função para enviar mensagens ao Discord 🚀


In [ ]:
def send_discord_message(text: str, webhook_url: str, username: str = "Assistente de Estudos", avatar_url: str = None):
    """
    Envia uma mensagem de texto para o Discord usando um webhook.
    """
    if not webhook_url or "discord.com/api/webhooks" not in webhook_url:
        print(f"ERRO: Discord Webhook URL inválido ou não configurado. Mensagem não enviada: '{text}'")
        return

    payload = {
        "content": text,
        "username": username,
        "avatar_url": avatar_url
    }
    try:
        response = requests.post(webhook_url, json=payload)
        response.raise_for_status() # Lança exceção para erros HTTP (4xx ou 5xx)
        print(f"\n[INTEGRAÇÃO DISCORD] Mensagem enviada com sucesso para o Discord!")
    except requests.exceptions.RequestException as e:
        print(f"\n[ERRO DISCORD] Falha ao enviar mensagem para o Discord: {e}")

### CÉLULA 7: Célula de Testes com Feedbacks


In [ ]:
# Célula de Testes com Materiais de Estudo
if __name__ == "__main__":

    # Exemplo 1: Slide de Biologia
    slide_biologia = slide_biologia = """FOTOSSÍNTESE
Processo pelo qual plantas, algas e algumas bactérias convertem energia luminosa em energia química.
Equação geral: 6CO2 + 6H2O + luz → C6H12O6 + 6O2
Ocorre nos cloroplastos, especificamente nos tilacoides.
Duas fases principais: fase luminosa (produz ATP e NADPH) e fase escura (ciclo de Calvin).
A clorofila é o pigmento principal responsável pela absorção de luz."""


    print(f"\nSlide Original (Biologia): {slide_biologia}")

    # Passo 1: Resumo do Slide
    resumo_biologia = slide_summary_chain.invoke({"texto_do_slide": slide_biologia}).strip()
    print(f"\nResumo: {resumo_biologia}")

    # Passo 2: Conceitos-Chave
    conceitos_biologia = key_concepts_chain.invoke({"texto_do_slide": slide_biologia}).strip()
    print(f"\nConceitos-Chave: {conceitos_biologia}")

    # Passo 3: Perguntas de Estudo
    perguntas_biologia = study_questions_chain.invoke({"texto_do_slide": slide_biologia}).strip()
    print(f"\nPerguntas de Estudo: {perguntas_biologia}")

    # Enviar para o Discord
    discord_message_biologia = (
        f"**NOVO RESUMO DE ESTUDO (Biologia)**\n"
        f"> **Slide Original:**\n```\n{slide_biologia}\n```\n"
        f"> **Resumo:**\n```\n{resumo_biologia}\n```\n"
        f"> **Conceitos-Chave:**\n```\n{conceitos_biologia}\n```\n"
        f"> **Perguntas de Estudo:**\n```\n{perguntas_biologia}\n```"
    )
    send_discord_message(discord_message_biologia, "https://discord.com/api/webhooks/1379881762196295680/LDoW4v5NWgPDLec6JTJ5fmgz4gY9rgOsP_6r62RqR86uFsfhNXa3TPf1h21U9FSMMozd", username="Assistente de Estudos")

    print("\n" + "="*50 + "\n")

    # Exemplo 2: Slide de História
    slide_historia = """REVOLUÇÃO INDUSTRIAL
    Período de grandes transformações econômicas e sociais entre 1760-1840.
    Origem: Inglaterra, com a mecanização da indústria têxtil.
    Principais invenções: máquina a vapor (James Watt), tear mecânico.
    Consequências: urbanização, surgimento do proletariado, capitalismo industrial.
    Impacto ambiental: início da poluição industrial em larga escala."""

    print(f"Slide Original (História): {slide_historia}")

    resumo_historia = slide_summary_chain.invoke({"texto_do_slide": slide_historia}).strip()
    print(f"\nResumo: {resumo_historia}")

    conceitos_historia = key_concepts_chain.invoke({"texto_do_slide": slide_historia}).strip()
    print(f"\nConceitos-Chave: {conceitos_historia}")

    perguntas_historia = study_questions_chain.invoke({"texto_do_slide": slide_historia}).strip()
    print(f"\nPerguntas de Estudo: {perguntas_historia}")

    discord_message_historia = (
        f"**NOVO RESUMO DE ESTUDO (História)**\n"
        f"> **Slide Original:**\n```\n{slide_historia}\n```\n"
        f"> **Resumo:**\n```\n{resumo_historia}\n```\n"
        f"> **Conceitos-Chave:**\n```\n{conceitos_historia}\n```\n"
        f"> **Perguntas de Estudo:**\n```\n{perguntas_historia}\n```"
    )
    send_discord_message(discord_message_historia, DISCORD_WEBHOOK_URL, username="Assistente de Estudos")


Slide Original (Biologia): FOTOSSÍNTESE
Processo pelo qual plantas, algas e algumas bactérias convertem energia luminosa em energia química.
Equação geral: 6CO2 + 6H2O + luz → C6H12O6 + 6O2
Ocorre nos cloroplastos, especificamente nos tilacoides.
Duas fases principais: fase luminosa (produz ATP e NADPH) e fase escura (ciclo de Calvin).
A clorofila é o pigmento principal responsável pela absorção de luz.

Resumo: Fotosíntese é o processo pelo qual plantas, algas e algumas bactérias convertem energia luminosa em energia química. Isso ocorre nos cloroplastos, especificamente nos tilacoides. A fotossíntese pode ser dividida em duas fases principais: fase luminosa e fase escura. A clorofila é o pigmento

Conceitos-Chave: - Fotosíntese
- Cloroplastos
- Tilacoides
- Equação geral de fotossíntese
- Fase luminosa (produz ATP e NADPH)
- Fase escura (ciclo de Calvin)
- Clorofila (pigmento principal responsável pela absorção de luz)

Perguntas de Estudo: 1. O que é a fotossíntese e como ela conve